In [ ]:
import scipy
import skimage
from skimage import io
from skimage import exposure
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from numpy import matlib
import seaborn as sns
import math
import os
import scipy.optimize
import scipy.io as sio
from scipy import stats
import scipy.ndimage
# import h5py
from skimage.morphology import disk
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
import matplotlib.patches as patches
from matplotlib import cm, colors
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
path='/Volumes/data_ssd2/Rojas_Lab/data'
expt_id = '/250609_bFB323_10uM_IPTG_tun_TIRF_v2'
cond='90min_HADA'
threshold=0.00012
scene_nums=6
condition=['$PBP1-mNeonGreen$ tunicamycin, 6/9/25']

# expt_id = '/250609_bFB323_10uM_IPTG_Tun_TIRF_v2'
# conds=['10uM_IPTG', '0min','10min','20min','30min','50min','70min','90min']
#
# thresholds=list(0.00012*np.ones(len(conds)))
# timepoints=[-5.0, 0.0, 10.0,20.0,30.0, 50.0,70.0,90.0]
# scene_nums=[3,2,2,2,2,2,2,2]
# condition=['$PBP1-mNeonGreen$ tunicamycin, 6/9/25 v2']

In [ ]:
scene=4
ims=[]
for channel in np.arange(2,4):
    ims.append(plt.imread(path+expt_id+'/'+cond+'/C{0}'.format(channel)+expt_id+'_'+cond+'_s{0}_C{1}.tif'.format(str(scene).zfill(3),channel)))

In [ ]:
fig=plt.figure(figsize=[14,7])
i=0
for im in ims:
    ax=plt.subplot(1,2,1+i)
    plt.imshow(im)
    i+=1

In [ ]:
sns.displot(ims[0].flatten())

In [ ]:
temp_im=skimage.filters.gaussian(ims[0], sigma=1.0)
sns.displot(temp_im.flatten()*np.amax(ims[0])/np.amax(temp_im))
temp_im*=np.amax(ims[0])/np.amax(temp_im)

In [ ]:
cutoff=1700
fig=plt.figure(figsize=[14,14])
ax=plt.subplot(2,2,1)
plt.imshow(ims[0])
ax=plt.subplot(2,2,2)
plt.imshow(ims[0]>cutoff)
ax=plt.subplot(2,2,3)
plt.imshow(temp_im)
ax=plt.subplot(2,2,4)
plt.imshow(temp_im>cutoff)


In [ ]:
coords=np.nonzero(ims[0]>cutoff)
xv=ims[0][coords]
yv=ims[1][coords]
plt.plot(xv,yv,linestyle='',alpha=0.5,marker=',')
plt.xlabel('PBP1')
plt.ylabel('HADA')

In [ ]:
coords=np.nonzero(ims[0]>cutoff)
xv=ims[0].flatten()
yv=ims[1].flatten()
plt.plot(xv,yv,linestyle='',alpha=0.5,marker=',')
plt.xlabel('PBP1')
plt.ylabel('HADA')

Okay, so there is indeed a correlation, but that doesn't really tell us much because it could be an artifact. Let's try spot detection.

In [ ]:
 # parameters for spot detection
min_sig = 2.0
max_sig = 4
ovrlap = 0.0
thresh=0.05
size=200
input_data = ims[0]/ np.amax([ims[0]])
out = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                   threshold=thresh, overlap=ovrlap, log_scale=False)
fig=plt.figure(figsize=[14,8])
plt.subplot(1,2,1)
plt.imshow(ims[0][:size,:size])
plt.subplot(1,2,2)
plt.imshow(ims[0][:size,:size])
yv,xv=np.array([int(val[0]) for val in out]), np.array([int(val[1]) for val in out])
# plt.plot(xv,yv,linestyle='',marker='x', color='w')
ax=plt.gca()
for temp_ind in np.arange(len(out)):
    # Define the circle's properties
    center_x = xv[temp_ind]
    center_y = yv[temp_ind]
    radius = np.sqrt(2)*out[temp_ind,2]
    # Create the Circle patch object
    circle = patches.Circle((center_x, center_y), radius, 
                        edgecolor='w', facecolor='None', linewidth=1)
    # Add the circle to the axes
    ax.add_patch(circle)
plt.title('Cell spots={0}'.format(len(out)),fontsize=10.0)
# Now we look at the cell periphery

In [ ]:
 # parameters for spot detection
min_sig = 2.0
max_sig = 4
ovrlap = 0.0
thresh=0.05
size=200
ind=1
input_data = ims[ind]/ np.amax([ims[ind]])
fig=plt.figure(figsize=[14,8])
plt.subplot(1,2,1)
plt.imshow(ims[ind][:size,:size])
ax=plt.gca()
plt.subplot(1,2,2)
ax=plt.gca()
plt.imshow(ims[ind][:size,:size])
for temp_ind in np.arange(len(out)):
    # Define the circle's properties
    center_x = xv[temp_ind]
    center_y = yv[temp_ind]
    radius = np.sqrt(2)*out[temp_ind,2]
    # Create the Circle patch object
    circle = patches.Circle((center_x, center_y), radius, 
                        edgecolor='r', facecolor='None', linewidth=1)
    # Add the circle to the axes
    ax.add_patch(circle)

out1 = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                   threshold=thresh, overlap=ovrlap, log_scale=False)
yv1,xv1=np.array([int(val[0]) for val in out1]), np.array([int(val[1]) for val in out1])
# plt.plot(xv,yv,linestyle='',marker='x', color='w')
for temp_ind in np.arange(len(out1)):
    # Define the circle's properties
    center_x = xv1[temp_ind]
    center_y = yv1[temp_ind]
    radius = np.sqrt(2)*out1[temp_ind,2]
    # Create the Circle patch object
    circle = patches.Circle((center_x, center_y), radius, 
                        edgecolor='w', facecolor='None', linewidth=1)
    # Add the circle to the axes
    ax.add_patch(circle)
plt.title('Cell spots={0}'.format(len(out1)),fontsize=10.0)
# Now we look at the cell periphery

In [ ]:
 # parameters for spot detection
min_sig = 2.0
max_sig = 4
ovrlap = 0.0
thresh=0.05
size=200
ind=1
input_data = ims[ind]/ np.amax([ims[ind]])
fig=plt.figure(figsize=[14,8])
plt.subplot(1,2,1)
plt.imshow(ims[0][:size,:size])
ax=plt.gca()
for temp_ind in np.arange(len(out)):
    # Define the circle's properties
    center_x = xv[temp_ind]
    center_y = yv[temp_ind]
    radius = np.sqrt(2)*out[temp_ind,2]
    # Create the Circle patch object
    circle = patches.Circle((center_x, center_y), radius, 
                        edgecolor='r', facecolor='None', linewidth=1)
    # Add the circle to the axes
    ax.add_patch(circle)
plt.title('PBP1 fluorescence')
plt.subplot(1,2,2)
plt.title('HADA stain')
ax=plt.gca()
plt.imshow(ims[ind][:size,:size])
for temp_ind in np.arange(len(out)):
    # Define the circle's properties
    center_x = xv[temp_ind]
    center_y = yv[temp_ind]
    radius = np.sqrt(2)*out[temp_ind,2]
    # Create the Circle patch object
    circle = patches.Circle((center_x, center_y), radius, 
                        edgecolor='r', facecolor='None', linewidth=1)
    # Add the circle to the axes
    ax.add_patch(circle)

out1 = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                   threshold=thresh, overlap=ovrlap, log_scale=False)
yv1,xv1=np.array([int(val[0]) for val in out1]), np.array([int(val[1]) for val in out1])
# plt.plot(xv,yv,linestyle='',marker='x', color='w')
for temp_ind in np.arange(len(out1)):
    # Define the circle's properties
    center_x = xv1[temp_ind]
    center_y = yv1[temp_ind]
    radius = np.sqrt(2)*out1[temp_ind,2]
    # Create the Circle patch object
    circle = patches.Circle((center_x, center_y), radius, 
                        edgecolor='w', facecolor='None', linewidth=1)
    # Add the circle to the axes
    ax.add_patch(circle)
# plt.title('Cell spots={0}'.format(len(out1)),fontsize=10.0)
fig.savefig('./outputs/compiled_data/staining_plots/HADA_PBP1_spot_comp.pdf',bbox_inches='tight')
# Now we look at the cell periphery

This already looks pretty good! To confirm our suspicions, let's calculate the fraction of red blobs that overlap with at least one white blob, then we'll randomly position the white blobs within the cell mask and use that to determine what the baseline overlap should be.

In [ ]:
coords_red=np.zeros([len(out),2])
radii=np.zeros([len(out),len(out1)])
for ind in range(len(out)):
    coords_red[ind,:]=out[ind][:2]
coords_white=np.zeros([len(out1),2])
for ind in range(len(out1)):
    coords_white[ind,:]=out1[ind][:2]
dist_map=euclidean_distances(coords_red,coords_white)
for ind in range(len(out)):
    for ind1 in range(len(out1)):
        radii[ind,ind1]=np.sqrt(2)*(out[ind][2]+out1[ind1][2])
contacts=(dist_map/radii)<1.0
num_cont=np.sum(np.sum(contacts,axis=1)>0)

In [ ]:
pos_cont=np.nonzero(np.sum(contacts,axis=1)>0)

In [ ]:
ind=1
fig=plt.figure(figsize=[14,8])
plt.subplot(1,2,1)
plt.imshow(ims[ind][:size,:size])
ax=plt.gca()
plt.subplot(1,2,2)
plt.imshow(ims[1][:size,:size])
ax=plt.gca()
for temp_ind in pos_cont[0]:
    # Define the circle's properties
    center_x = xv[temp_ind]
    center_y = yv[temp_ind]
    radius = np.sqrt(2)*out[temp_ind,2]
    # Create the Circle patch object
    circle = patches.Circle((center_x, center_y), radius, 
                        edgecolor='r', facecolor='None', linewidth=1)
    # Add the circle to the axes
    ax.add_patch(circle)

out1 = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                   threshold=thresh, overlap=ovrlap, log_scale=False)
yv1,xv1=np.array([int(val[0]) for val in out1]), np.array([int(val[1]) for val in out1])
# plt.plot(xv,yv,linestyle='',marker='x', color='w')
for temp_ind in np.arange(len(out1)):
    # Define the circle's properties
    center_x = xv1[temp_ind]
    center_y = yv1[temp_ind]
    radius = np.sqrt(2)*out1[temp_ind,2]
    # Create the Circle patch object
    circle = patches.Circle((center_x, center_y), radius, 
                        edgecolor='w', facecolor='None', linewidth=1)
    # Add the circle to the axes
    ax.add_patch(circle)
plt.title('Cell spots={0}'.format(len(out1)),fontsize=10.0)
# Now we look at the cell periphery

In [ ]:
print('Fraction with overlap:', len(pos_cont[0])/len(out)*1.0)

Okay, this looks like it's working!! Now, let's see what the fraction is for all scenes

In [ ]:
frac_overlap=[]
 # parameters for spot detection
min_sig = 2.0
max_sig = 4
ovrlap = 0.0
thresh=0.05
size=200

for scene in np.arange(1,scene_nums+1):
    ims=[]
    #Importing our data
    for channel in np.arange(2,4):
        ims.append(plt.imread(path+expt_id+'/'+cond+'/C{0}'.format(channel)+expt_id+'_'+cond+'_s{0}_C{1}.tif'.format(str(scene).zfill(3),channel)))
    # Analysing our PBP1 data
    input_data = ims[0]/ np.amax([ims[0]])
    out = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                       threshold=thresh, overlap=ovrlap, log_scale=False)
    yv,xv=np.array([int(val[0]) for val in out]), np.array([int(val[1]) for val in out])
    # Analysing our HADA data
    input_data = ims[1]/ np.amax([ims[1]])
    out1 = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                       threshold=thresh, overlap=ovrlap, log_scale=False)
    
    yv1,xv1=np.array([int(val[0]) for val in out1]), np.array([int(val[1]) for val in out1])

    coords_red=np.zeros([len(out),2])
    radii=np.zeros([len(out),len(out1)])
    for ind in range(len(out)):
        coords_red[ind,:]=out[ind][:2]
    coords_white=np.zeros([len(out1),2])
    for ind in range(len(out1)):
        coords_white[ind,:]=out1[ind][:2]
    dist_map=euclidean_distances(coords_red,coords_white)
    for ind in range(len(out)):
        for ind1 in range(len(out1)):
            radii[ind,ind1]=np.sqrt(2)*(out[ind][2]+out1[ind1][2])
    contacts=(dist_map/radii)<1.0
    pos_cont=np.nonzero(np.sum(contacts,axis=1)>0)
    frac_overlap.append(len(pos_cont[0])/len(out)*1.0)
    print('Done scene ', scene)
    

In [ ]:
frac_overlap

In [ ]:
path='/Volumes/data_ssd2/Rojas_Lab/data'
expt_ids = ['/250609_bFB323_10uM_IPTG_tun_TIRF_v2','/250609_bFB323_10uM_IPTG_tun_TIRF']
cond='90min_HADA'
scene_nums=[6,7]

frac_overlap=[]
 # parameters for spot detection
min_sig = 2.0
max_sig = 4
ovrlap = 0.0
thresh=0.05
size=200

num_PBP1_spots=[]
num_HADA_spots=[]
for expt_id in expt_ids:
    for scene in np.arange(1,scene_nums[expt_ids.index(expt_id)]+1):
        ims=[]
        #Importing our data
        for channel in np.arange(2,4):
            ims.append(plt.imread(path+expt_id+'/'+cond+'/C{0}'.format(channel)+expt_id+'_'+cond+'_s{0}_C{1}.tif'.format(str(scene).zfill(3),channel)))
        # Analysing our PBP1 data
        input_data = ims[0]/ np.amax([ims[0]])
        out = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                           threshold=thresh, overlap=ovrlap, log_scale=False)
        yv,xv=np.array([int(val[0]) for val in out]), np.array([int(val[1]) for val in out])
        # Analysing our HADA data
        input_data = ims[1]/ np.amax([ims[1]])
        out1 = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                           threshold=thresh, overlap=ovrlap, log_scale=False)
        
        yv1,xv1=np.array([int(val[0]) for val in out1]), np.array([int(val[1]) for val in out1])
    
        coords_red=np.zeros([len(out),2])
        radii=np.zeros([len(out),len(out1)])
        for ind in range(len(out)):
            coords_red[ind,:]=out[ind][:2]
        coords_white=np.zeros([len(out1),2])
        for ind in range(len(out1)):
            coords_white[ind,:]=out1[ind][:2]
        dist_map=euclidean_distances(coords_red,coords_white)
        for ind in range(len(out)):
            for ind1 in range(len(out1)):
                radii[ind,ind1]=np.sqrt(2)*(out[ind][2]+out1[ind1][2])
        contacts=(dist_map/radii)<1.0
        pos_cont=np.nonzero(np.sum(contacts,axis=1)>0)
        frac_overlap.append(len(pos_cont[0])/len(out)*1.0)
        print('Done scene ', scene, expt_id)
        print(len(out),len(out1))
        num_PBP1_spots.append(len(out))
        num_HADA_spots.append(len(out1))

In [ ]:
print(sum(num_PBP1_spots))
print(sum(num_HADA_spots))

In [ ]:
np.mean(frac_overlap)

In [ ]:
np.std(frac_overlap)

Okay, that sounds great in principle. Just what should our baseline expectation be? Also, can we save images that show that this spot detection is working in all cases?

In [ ]:
for channel in np.arange(2,4):
    ims.append(plt.imread(path+expt_id+'/'+cond+'/C{0}'.format(channel)+expt_id+'_'+cond+'_s{0}_C{1}.tif'.format(str(scene).zfill(3),channel)))
temp_im=skimage.filters.gaussian(ims[0], sigma=1.0)
temp_im*=np.amax(ims[0])/np.amax(temp_im)
coords_mask=np.nonzero(temp_im>cutoff)

In [ ]:
print(expt_id,scene)
cutoff=1700
rep_num=1
ims=[]
#Importing our data
for channel in np.arange(2,4):
    ims.append(plt.imread(path+expt_id+'/'+cond+'/C{0}'.format(channel)+expt_id+'_'+cond+'_s{0}_C{1}.tif'.format(str(scene).zfill(3),channel)))
temp_im=skimage.filters.gaussian(ims[0], sigma=1.0)
temp_im*=np.amax(ims[0])/np.amax(temp_im)
coords_mask=np.nonzero(temp_im>cutoff)
out = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                   threshold=thresh, overlap=ovrlap, log_scale=False)
yv,xv=np.array([int(val[0]) for val in out]), np.array([int(val[1]) for val in out])
# Analysing our HADA data
input_data = ims[1]/ np.amax([ims[1]])
out1 = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                   threshold=thresh, overlap=ovrlap, log_scale=False)
yv1,xv1=np.array([int(val[0]) for val in out1]), np.array([int(val[1]) for val in out1])
rads=np.zeros(len(out))
for ind in range(len(out)):
    rads[ind]=np.sqrt(2)*out[ind][2]

for rep in range(rep_num):
    temp_int=np.random.randint(low=0,high=len(coords_mask[0]),size=1)[0]
    # print(temp_int)
    yv_rand=np.array([[coords_mask[0][temp_int]]])
    xv_rand=np.array([[coords_mask[1][temp_int]]])
    while len(xv_rand)<len(out):
        temp_int=np.random.randint(low=0,high=len(coords_mask[0]),size=1)[0]
        # print(temp_int)
        coords_existing=np.concatenate((xv_rand,yv_rand),axis=1)
        temp_dists=euclidean_distances(coords_existing,np.array([[coords_mask[1][temp_int],coords_mask[0][temp_int]]]))[:,0]
        temp_dists*=1.0/(rads[xv_rand.shape[0]]+rads[:xv_rand.shape[0]])
        if np.sum(temp_dists<1.0)==0:
            yv_rand=np.concatenate((yv_rand, np.array([[coords_mask[0][temp_int]]])),axis=0)
            xv_rand=np.concatenate((xv_rand, np.array([[coords_mask[1][temp_int]]])),axis=0)
    print("rep", rep)
fig=plt.figure(figsize=[6,6])
plt.imshow(temp_im[:200,:200])
ax=plt.gca()
for temp_ind in np.arange(len(out)):
    # Define the circle's properties
    center_x = xv_rand[temp_ind]
    center_y = yv_rand[temp_ind]
    radius = rads[temp_ind]
    # Create the Circle patch object
    circle = patches.Circle((center_x, center_y), radius, 
                        edgecolor='r', facecolor='None', linewidth=1)
    # Add the circle to the axes
    ax.add_patch(circle)

for temp_ind in np.arange(len(out1)):
    # Define the circle's properties
    center_x = xv1[temp_ind]
    center_y = yv1[temp_ind]
    radius = np.sqrt(2)*out1[temp_ind,2]
    # Create the Circle patch object
    circle = patches.Circle((center_x, center_y), radius, 
                        edgecolor='w', facecolor='None', linewidth=1)
    # Add the circle to the axes
    ax.add_patch(circle)
plt.title('Cell spots={0}'.format(len(out1)),fontsize=10.0)

coords_red=np.zeros([len(out),2])
radii=np.zeros([len(out),len(out1)])
for ind in range(len(out)):
    coords_red[:,0]=yv_rand[:,0]
    coords_red[:,1]=xv_rand[:,0]
coords_white=np.zeros([len(out1),2])
for ind in range(len(out1)):
    coords_white[ind,:]=out1[ind][:2]
dist_map=euclidean_distances(coords_red,coords_white)
for ind in range(len(out)):
    for ind1 in range(len(out1)):
        radii[ind,ind1]=np.sqrt(2)*(out[ind][2]+out1[ind1][2])
contacts=(dist_map/radii)<1.0
pos_cont=np.nonzero(np.sum(contacts,axis=1)>0)
frac_overlap.append(len(pos_cont[0])/len(out)*1.0)

In [ ]:
print(expt_id,scene)
cutoff=1700
rep_num=100
ims=[]
#Importing our data
for channel in np.arange(2,4):
    ims.append(plt.imread(path+expt_id+'/'+cond+'/C{0}'.format(channel)+expt_id+'_'+cond+'_s{0}_C{1}.tif'.format(str(scene).zfill(3),channel)))
temp_im=skimage.filters.gaussian(ims[0], sigma=1.0)
temp_im*=np.amax(ims[0])/np.amax(temp_im)
coords_mask=np.nonzero(temp_im>cutoff)
out = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                   threshold=thresh, overlap=ovrlap, log_scale=False)
yv,xv=np.array([int(val[0]) for val in out]), np.array([int(val[1]) for val in out])
# Analysing our HADA data
input_data = ims[1]/ np.amax([ims[1]])
out1 = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                   threshold=thresh, overlap=ovrlap, log_scale=False)
yv1,xv1=np.array([int(val[0]) for val in out1]), np.array([int(val[1]) for val in out1])
rads=np.zeros(len(out))
for ind in range(len(out)):
    rads[ind]=np.sqrt(2)*out[ind][2]

frac_overlap1=[]
for rep in range(rep_num):
    temp_int=np.random.randint(low=0,high=len(coords_mask[0]),size=1)[0]
    # print(temp_int)
    yv_rand=np.array([[coords_mask[0][temp_int]]])
    xv_rand=np.array([[coords_mask[1][temp_int]]])
    while len(xv_rand)<len(out):
        temp_int=np.random.randint(low=0,high=len(coords_mask[0]),size=1)[0]
        # print(temp_int)
        coords_existing=np.concatenate((xv_rand,yv_rand),axis=1)
        temp_dists=euclidean_distances(coords_existing,np.array([[coords_mask[1][temp_int],coords_mask[0][temp_int]]]))[:,0]
        temp_dists*=1.0/(rads[xv_rand.shape[0]]+rads[:xv_rand.shape[0]])
        if np.sum(temp_dists<1.0)==0:
            yv_rand=np.concatenate((yv_rand, np.array([[coords_mask[0][temp_int]]])),axis=0)
            xv_rand=np.concatenate((xv_rand, np.array([[coords_mask[1][temp_int]]])),axis=0)
    # print("rep", rep)
    coords_red=np.zeros([len(out),2])
    radii=np.zeros([len(out),len(out1)])
    for ind in range(len(out)):
        coords_red[:,0]=yv_rand[:,0]
        coords_red[:,1]=xv_rand[:,0]
    coords_white=np.zeros([len(out1),2])
    for ind in range(len(out1)):
        coords_white[ind,:]=out1[ind][:2]
    dist_map=euclidean_distances(coords_red,coords_white)
    for ind in range(len(out)):
        for ind1 in range(len(out1)):
            radii[ind,ind1]=np.sqrt(2)*(out[ind][2]+out1[ind1][2])
    contacts=(dist_map/radii)<1.0
    pos_cont=np.nonzero(np.sum(contacts,axis=1)>0)
    frac_overlap1.append(len(pos_cont[0])/len(out)*1.0)
    if rep%10==0:
        print('Rep: ',rep)
print(np.mean(frac_overlap1))

Okay, so this is actually working now. Let's iterate through all scenes and experiments to get a fair sense of the relative increase in each scene.

In [ ]:
path='/Volumes/data_ssd2/Rojas_Lab/data'
expt_ids = ['/250609_bFB323_10uM_IPTG_tun_TIRF_v2','/250609_bFB323_10uM_IPTG_tun_TIRF']
cond='90min_HADA'
scene_nums=[6,7]

frac_overlap=[]
frac_overlap_bkgd=[]
 # parameters for spot detection
min_sig = 2.0
max_sig = 4
ovrlap = 0.0
thresh=0.05
size=200

cutoff=1700
rep_num=100

col=['Experiment', 'Scene', 'Overlap (%)', 'Type']
df=pd.DataFrame(columns=col)
for expt_id in expt_ids:
    for scene in np.arange(1,scene_nums[expt_ids.index(expt_id)]+1):
        ims=[]
        #Importing our data
        for channel in np.arange(2,4):
            ims.append(plt.imread(path+expt_id+'/'+cond+'/C{0}'.format(channel)+expt_id+'_'+cond+'_s{0}_C{1}.tif'.format(str(scene).zfill(3),channel)))
        # Analysing our PBP1 data
        input_data = ims[0]/ np.amax([ims[0]])
        out = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                           threshold=thresh, overlap=ovrlap, log_scale=False)
        yv,xv=np.array([int(val[0]) for val in out]), np.array([int(val[1]) for val in out])
        # Analysing our HADA data
        input_data = ims[1]/ np.amax([ims[1]])
        out1 = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                           threshold=thresh, overlap=ovrlap, log_scale=False)
        
        yv1,xv1=np.array([int(val[0]) for val in out1]), np.array([int(val[1]) for val in out1])
    
        coords_red=np.zeros([len(out),2])
        radii=np.zeros([len(out),len(out1)])
        for ind in range(len(out)):
            coords_red[ind,:]=out[ind][:2]
        coords_white=np.zeros([len(out1),2])
        for ind in range(len(out1)):
            coords_white[ind,:]=out1[ind][:2]
        dist_map=euclidean_distances(coords_red,coords_white)
        for ind in range(len(out)):
            for ind1 in range(len(out1)):
                radii[ind,ind1]=np.sqrt(2)*(out[ind][2]+out1[ind1][2])
        contacts=(dist_map/radii)<1.0
        pos_cont=np.nonzero(np.sum(contacts,axis=1)>0)
        frac_overlap.append(len(pos_cont[0])/len(out)*1.0)
        print('Done scene ', scene, expt_id)

        # Now we save a picture showing this
        fig=plt.figure(figsize=[14,8])
        plt.subplot(1,2,1)
        plt.imshow(ims[0][:size,:size])
        ax=plt.gca()
        for temp_ind in np.arange(len(out)):
            # Define the circle's properties
            center_x = xv[temp_ind]
            center_y = yv[temp_ind]
            radius = np.sqrt(2)*out[temp_ind,2]
            # Create the Circle patch object
            circle = patches.Circle((center_x, center_y), radius, 
                                edgecolor='g', facecolor='None', linewidth=1)
            # Add the circle to the axes
            ax.add_patch(circle)
        plt.title('PBP1 fluorescence')
        plt.subplot(1,2,2)
        plt.title('HADA stain')
        ax=plt.gca()
        plt.imshow(ims[1][:size,:size])
        for temp_ind in np.arange(len(out)):
            # Define the circle's properties
            center_x = xv[temp_ind]
            center_y = yv[temp_ind]
            radius = np.sqrt(2)*out[temp_ind,2]
            # Create the Circle patch object
            circle = patches.Circle((center_x, center_y), radius, 
                                edgecolor='g', facecolor='None', linewidth=1)
            # Add the circle to the axes
            ax.add_patch(circle)
        
        out1 = skimage.feature.blob_log(input_data, min_sigma=min_sig, max_sigma=max_sig, num_sigma=10,
                                                           threshold=thresh, overlap=ovrlap, log_scale=False)
        yv1,xv1=np.array([int(val[0]) for val in out1]), np.array([int(val[1]) for val in out1])
        # plt.plot(xv,yv,linestyle='',marker='x', color='w')
        for temp_ind in np.arange(len(out1)):
            # Define the circle's properties
            center_x = xv1[temp_ind]
            center_y = yv1[temp_ind]
            radius = np.sqrt(2)*out1[temp_ind,2]
            # Create the Circle patch object
            circle = patches.Circle((center_x, center_y), radius, 
                                edgecolor='w', facecolor='None', linewidth=1)
            # Add the circle to the axes
            ax.add_patch(circle)
        # plt.title('Cell spots={0}'.format(len(out1)),fontsize=10.0)
        fig.savefig('./outputs/compiled_data/FDAA_corr'+expt_id+'_scene_{0}_FDAA_corr.pdf'.format(scene),bbox_inches='tight')
        
        # Now we do the random background spot overlap test, randomly assigning these spots to positions within the mask
        temp_im=skimage.filters.gaussian(ims[0], sigma=1.0)
        temp_im*=np.amax(ims[0])/np.amax(temp_im)
        coords_mask=np.nonzero(temp_im>cutoff)
        
        rads=np.zeros(len(out))
        for ind in range(len(out)):
            rads[ind]=np.sqrt(2)*out[ind][2]
        
        frac_overlap1=[]
        for rep in range(rep_num):
            temp_int=np.random.randint(low=0,high=len(coords_mask[0]),size=1)[0]
            # print(temp_int)
            yv_rand=np.array([[coords_mask[0][temp_int]]])
            xv_rand=np.array([[coords_mask[1][temp_int]]])
            while len(xv_rand)<len(out):
                temp_int=np.random.randint(low=0,high=len(coords_mask[0]),size=1)[0]
                # print(temp_int)
                coords_existing=np.concatenate((xv_rand,yv_rand),axis=1)
                temp_dists=euclidean_distances(coords_existing,np.array([[coords_mask[1][temp_int],coords_mask[0][temp_int]]]))[:,0]
                temp_dists*=1.0/(rads[xv_rand.shape[0]]+rads[:xv_rand.shape[0]])
                if np.sum(temp_dists<1.0)==0:
                    yv_rand=np.concatenate((yv_rand, np.array([[coords_mask[0][temp_int]]])),axis=0)
                    xv_rand=np.concatenate((xv_rand, np.array([[coords_mask[1][temp_int]]])),axis=0)
            # print("rep", rep)
            coords_red=np.zeros([len(out),2])
            radii=np.zeros([len(out),len(out1)])
            for ind in range(len(out)):
                coords_red[:,0]=yv_rand[:,0]
                coords_red[:,1]=xv_rand[:,0]
            coords_white=np.zeros([len(out1),2])
            for ind in range(len(out1)):
                coords_white[ind,:]=out1[ind][:2]
            dist_map=euclidean_distances(coords_red,coords_white)
            for ind in range(len(out)):
                for ind1 in range(len(out1)):
                    radii[ind,ind1]=np.sqrt(2)*(out[ind][2]+out1[ind1][2])
            contacts=(dist_map/radii)<1.0
            pos_cont=np.nonzero(np.sum(contacts,axis=1)>0)
            frac_overlap1.append(len(pos_cont[0])/len(out)*1.0)
            if rep%10==0:
                print('Rep: ',rep)
        frac_overlap_bkgd.append(np.mean(frac_overlap1))
        temp_df=pd.DataFrame(data=[[expt_id,scene,frac_overlap[-1],'Data']],columns=col)
        df=pd.concat((df,temp_df))
        temp_df=pd.DataFrame(data=[[expt_id,scene,frac_overlap_bkgd[-1],'Null hyp.']],columns=col)
        df=pd.concat((df,temp_df))
        
print(expt_id,scene)

In [ ]:
df['Overlap frac'] = df['Overlap (%)']

In [ ]:
df['Overlap (%)']=100.0*df['Overlap frac']
fig=plt.figure(figsize=[1.5,2.0])
sns.set(font_scale=1.5)
sns.set_style('whitegrid')
sns.barplot(data=df,x='Type', y='Overlap (%)',capsize=0.5)
plt.xlabel(None)
plt.xticks(rotation=45)
plt.ylabel('% Overlapping')
fig.savefig('./outputs/compiled_data/FDAA_corr/overlapping_puncta.pdf',bbox_inches='tight')

In [ ]:
df.groupby('Type').describe()

In [ ]:
scipy.stats.ttest_ind(frac_overlap, frac_overlap_bkgd)